<a href="https://colab.research.google.com/github/LikithaVarma/Assignment/blob/main/likitha_flan_t5_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.10

In [ ]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
TOKENIZER = T5TokenizerFast.from_pretrained('google/flan-t5-small')
MODEL = T5ForConditionalGeneration.from_pretrained('google/flan-t5-small')
OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4
DEVICE = "cuda:0"

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2024-04-26 17:44:46--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2024-04-26 17:44:48 (348 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2024-04-26 17:44:48--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [ ]:
with open('train-v2.0.json', 'rb') as f:
  train_data = json.load(f)
with open('dev-v2.0.json', 'rb') as f:
  val_data = json.load(f)

In [ ]:
def prepare_data(data):
    articles = []

    for article in data["data"]:
        for paragraph in article["paragraphs"]:
            for qa in paragraph["qas"]:
                question = qa["question"]

                if not qa["is_impossible"]:
                  answer = qa["answers"][0]["text"]

                inputs = {"context": paragraph["context"], "question": question, "answer": answer}


                articles.append(inputs)

    return articles

In [ ]:
train_data = prepare_data(train_data)
val_data = prepare_data(val_data)
train_data = pd.DataFrame(train_data)
val_data = pd.DataFrame(val_data)

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

In [ ]:
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [ ]:
train_dataset = QA_Dataset(TOKENIZER, train_data, Q_LEN, T_LEN)
val_dataset = QA_Dataset(TOKENIZER, val_data, Q_LEN, T_LEN)

In [ ]:
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

In [ ]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=False
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.772300,1.460428,0.400707,0.238500,0.399651,0.399761


TrainOutput(global_step=16290, training_loss=1.03486622622439, metrics={'train_runtime': 4096.7241, 'train_samples_per_second': 31.811, 'train_steps_per_second': 3.976, 'total_flos': 1.2112532341260288e+16, 'train_loss': 1.03486622622439, 'epoch': 1.0})

In [ ]:
import os
model_path = '/content/drive/MyDrive/FLAN_T5-SQuAD'
if not os.path.exists(model_path):
  os.mkdir(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/FLAN_T5-SQuAD/tokenizer_config.json',
 '/content/drive/MyDrive/FLAN_T5-SQuAD/special_tokens_map.json',
 '/content/drive/MyDrive/FLAN_T5-SQuAD/spiece.model',
 '/content/drive/MyDrive/FLAN_T5-SQuAD/added_tokens.json')

In [ ]:
model = model.to(DEVICE)

In [ ]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                             references=[ref_answer])

        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer

In [ ]:
context = val_data.iloc[0]["context"]
question = val_data.iloc[0]["question"]
answer = val_data.iloc[0]["answer"]

predict_answer(context,question,answer)

Context: 
 The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.


Question: 
 In what country is Normandy located?


{'Reference Answer: ': 'France',
 'Predicted Answer: ': 'France',
 'BLEU Score: ': {'google_bleu': 1.0}}

In [ ]:
context = "In recent times, the COVID-19 pandemic has had a profound impact on global health and society. Understanding the symptoms associated with the disease is crucial for early detection and appropriate management. COVID-19 typically manifests with a range of symptoms, which may include fever, cough, shortness of breath, fatigue, body aches, loss of taste or smell, sore throat, congestion, nausea, vomiting, and diarrhea. However, it's important to note that symptoms can vary widely among individuals, and some people infected with the virus may remain asymptomatic. Stay informed and vigilant about the latest guidance from health authorities to protect yourself and others from the spread of COVID-19."
question = "What are the common symptoms of COVID-19?"
answer = "The common symptoms of COVID-19 are fever, cough, shortness of breath, fatigue, body aches, loss of taste or smell, sore throat, congestion, nausea, vomiting, and diarrhea."

predict_answer(context,question,answer)

Context: 
 In recent times, the COVID-19 pandemic has had a profound impact on global health and society. Understanding the symptoms associated with the disease is crucial for early detection and appropriate management. COVID-19 typically manifests with a range of symptoms, which may include fever, cough, shortness of breath, fatigue, body aches, loss of taste or smell, sore throat, congestion, nausea, vomiting, and diarrhea. However, it's important to note that symptoms can vary widely among individuals, and some people infected with the virus may remain asymptomatic. Stay informed and vigilant about the latest guidance from health authorities to protect yourself and others from the spread of COVID-19.


Question: 
 What are the common symptoms of COVID-19?


{'Reference Answer: ': 'The common symptoms of COVID-19 are fever, cough, shortness of breath, fatigue, body aches, loss of taste or smell, sore throat, congestion, nausea, vomiting, and diarrhea.',
 'Predicted Answer: ': 'fever, cough, shortness of breath, fatigue, body aches, loss of taste or',
 'BLEU Score: ': {'google_bleu': 0.43661971830985913}}

In [ ]:
with open('/content/drive/MyDrive/flan_t5/sample.json', 'rb') as f:
  test_data = json.load(f)
test_data = pd.DataFrame(test_data)

In [ ]:
for i in range(5):
  curr = test_data.iloc[i]
  print(predict_answer(curr['context'],curr['question'],curr['answer']))

Context: 
 The Great Wall of China is one of the most remarkable architectural feats in history. It stretches over 13,000 miles and was built over several centuries to protect China from invasions.


Question: 
 What is the purpose of the Great Wall of China?
{'Reference Answer: ': 'To protect China from invasions.', 'Predicted Answer: ': 'to protect China from invasions', 'BLEU Score: ': {'google_bleu': 0.5555555555555556}}
Context: 
 Marie Curie was a pioneering scientist known for her research on radioactivity. She was the first woman to win a Nobel Prize and the only person to win Nobel Prizes in two different scientific fields.


Question: 
 What was Marie Curie famous for?
{'Reference Answer: ': 'Her research on radioactivity and being the first woman to win a Nobel Prize.', 'Predicted Answer: ': 'research on radioactivity', 'BLEU Score: ': {'google_bleu': 0.1111111111111111}}
Context: 
 The Theory of Relativity, proposed by Albert Einstein in the early 20th century, revolutioniz